In [5]:
import numpy as np
import pandas as pd
import scipy.stats as st
from pandas.core.dtypes.common import is_numeric_dtype

data = pd.read_csv("finalData.csv")
data = data.set_index('projectID')
pd.set_option('display.float_format',  '{:,.3f}'.format)
data

,success,Lamount,creator_gender,Lgoal,words,backers_count,serial_entrepreneur,duration,category_art,category_comics,...,category_photography,category_publishing,category_technology,category_theater,month,year,media,sustainability,story,url
projectID,,,,,,,,,,,,,,,,,,,,,
3731940,0,2.307,0,3.778,71,5,0,60.000,0,0,...,0,0,1,0,6,2019,0,0,app go benefit trainer personnel look get fit ...,https://www.kickstarter.com/projects/fitnesspo...
3451605,1,5.067,0,4.000,497,2003,1,14.583,0,0,...,0,0,0,0,6,2019,1,0,even miss kickstarter still get hand papillon ...,https://www.kickstarter.com/projects/kolossalg...
3722583,0,1.447,0,3.778,443,6,0,30.000,0,1,...,0,0,0,0,6,2019,1,0,journey blak blak blak officially entitle blak...,https://www.kickstarter.com/projects/myheart/b...
3745862,1,3.266,1,3.176,474,48,0,39.687,0,0,...,0,0,0,0,6,2019,1,0,isabel jazz folk musical teach empathy metoo s...,https://www.kickstarter.com/projects/annawestb...
3735376,1,3.000,1,3.000,130,17,0,25.990,0,0,...,0,0,0,1,6,2019,0,0,main stage kids head new york young performer ...,https://www.kickstarter.com/projects/mainstage...
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
4378823,1,4.797,0,4.447,837,232,1,30.000,0,0,...,1,0,0,0,5,2022,1,0,book maui mauka makai companion book maui coas...,https://www.kickstarter.com/projects/danielsul...
4394019,1,4.045,0,4.000,167,80,0,26.377,0,0,...,0,0,0,0,5,2022,1,0,myrcene ale co dream mike goergen cy higgins m...,https://www.kickstarter.com/projects/myrceneal...
4385597,1,4.403,0,4.398,138,73,0,14.147,0,0,...,0,0,0,1,5,2022,1,0,firefly theatrical need help build theatre spa...,https://www.kickstarter.com/projects/fireflyth...


In [6]:
# Table 1
def getCategory(entry):
    for category in entry.index:
        if entry.get(category) == 1:
            return category.split("_")[1]

    return "unknown"


table1 = data.copy()
table1['Category'] = table1.apply(lambda entry: getCategory(entry[8:23]), axis=1)
table1 = table1.rename(columns={"creator_gender": "Female"})
table1 = table1.groupby(['Category', 'Female']).agg({'success': ['size', 'mean', 'std']})
table1.columns = ['Obs', 'Mean', 'SD']
table1

Obs  Mean    SD
Category    Female                  
art         0        376 0.809 0.394
            1        643 0.865 0.342
comics      0        791 0.934 0.248
            1        147 0.884 0.321
crafts      0         69 0.565 0.499
            1         86 0.512 0.503
dance       0          4 0.750 0.500
            1         13 0.846 0.376
design      0        439 0.945 0.228
            1        168 0.905 0.294
fashion     0        192 0.578 0.495
            1        246 0.809 0.394
film&video  0        511 0.542 0.499
            1        199 0.623 0.486
food        0        303 0.482 0.500
            1        204 0.422 0.495
games       0       1243 0.926 0.262
            1        210 0.900 0.301
journalism  0         41 0.415 0.499
            1         31 0.226 0.425
music       0        474 0.781 0.414
            1        198 0.823 0.382
photography 0         74 0.689 0.466
            1         40 0.675 0.474
publishing  0        531 0.846 0.362
            1        393 0.888 0.316
technology  0        550 0.367 0.483
            1        133 0.308 0.464
theater     0         46 0.630 0.488
            1         33 0.697 0.467

In [7]:
# Table 2
table2 = data.copy()
table2 = table2.rename(columns={"creator_gender": "Female"})
table2 = table2.groupby(['Female']).agg({'success': ['size', 'mean', 'std']})
table2.columns = ['Obs', 'Mean', 'SD']

# maleData = data[data['creator_gender'] == 0]
# maleCI = st.norm.interval(confidence=0.95,
#                           loc=np.mean(maleData['success']),
#                           scale=st.sem(maleData['success']))
# femaleData = data[data['creator_gender'] == 1]
# femaleCI = st.norm.interval(confidence=0.95,
#                             loc=np.mean(femaleData['success']),
#                             scale=st.sem(femaleData['success']))
# table2.insert(2, "95% CI (lower)", [maleCI[0], femaleCI[0]])
# table2.insert(3, "95% CI (upper)", [maleCI[1], femaleCI[1]])

table2

,Obs,Mean,SD
Female,,,
0,5644,0.762,0.426
1,2744,0.766,0.424


In [8]:
# Table 4
variableNames = []
means = []
medians = []
minima = []
maxima = []
SDs = []

for variableName in data.columns:
    variableData = data[variableName]
    if is_numeric_dtype(variableData):
        if variableName[0] == "L":
            variableData = 10**variableData
            variableName = variableName.replace("L", "")
        variableNames.append(variableName)
        means.append(np.mean(variableData))
        medians.append(np.median(variableData))
        minima.append(np.min(variableData))
        maxima.append(np.max(variableData))
        SDs.append(np.std(variableData))

variableNames[variableNames.index("creator_gender")] = "female"
table3 = {"Variable": variableNames, "Mean": means, "Median": medians, "Minimum": minima, "Maximum": maxima, "SD": SDs}
table3 = pd.DataFrame.from_dict(table3)
table3 = table3.set_index("Variable")
table3

,Mean,Median,Minimum,Maximum,SD
Variable,,,,,
success,0.763,1.000,0.000,1.000,0.425
amount,"66,149.066","3,838.000",1.000,"272,707,148.204","2,984,093.960"
female,0.327,0.000,0.000,1.000,0.469
goal,"19,854.147","4,000.000",1.000,"14,000,000.000","188,831.801"
words,408.228,305.000,2.000,"2,588.000",362.641
backers_count,345.802,65.000,1.000,"55,024.000","1,595.292"
serial_entrepreneur,0.252,0.000,0.000,1.000,0.434
duration,32.190,30.000,1.000,73.958,12.615
category_art,0.121,0.000,0.000,1.000,0.327
